In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testdate/in.txt
/kaggle/input/couplet/vocab.bin
/kaggle/input/couplet/encoder.json
/kaggle/input/couplet/decoder.json


In [9]:
import torch
import torch.nn as nn

# 编码器
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, dropout):
        super(Encoder, self).__init__()
        # 定义嵌入层
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # 定义GRU层
        self.rnn = nn.GRU(emb_dim, hidden_dim,dropout=dropout, 
                          batch_first=True, bidirectional=True)

    def forward(self, token_seq):
        # token_seq: [batch_size, seq_len]
        # embedded: [batch_size, seq_len, emb_dim]
        embedded = self.embedding(token_seq)

        # outputs: [batch_size, seq_len, hidden_dim * 2]
        # hidden: [2, batch_size, hidden_dim]
        outputs, hidden = self.rnn(embedded)

        # 返回，Encoder最后一个时间步的隐藏状态(拼接)
        # return outputs[:, -1, :]
        # 返回最后一个时间步的隐藏状态(拼接)
        return torch.cat((hidden[0], hidden[1]), dim=1), outputs
        # 返回最后一个时间步的隐状态（相加）
        # return hidden.sum(dim=0)

# Attention层
class Attention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, enc_output, dec_output):
        # a_t = h_t @ h_s  
        a_t = torch.bmm(enc_output, dec_output.permute(0, 2, 1))
        # 1.计算 结合解码token和编码token，关联的权重
        a_t = torch.softmax(a_t, dim=1)
        # 2.计算 关联权重和编码token 贡献值
        c_t = torch.bmm(a_t.permute(0, 2, 1), enc_output)
        return c_t

# 解码器
class Decoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, dropout):
        super(Decoder, self).__init__()
        # 定义嵌入层
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # 定义GRU层
        self.rnn = nn.GRU(emb_dim, hidden_dim*2 , dropout=dropout,
                          batch_first=True)
        # 定义线性层
        self.fc = nn.Linear(hidden_dim*2 , input_dim)  # 解码词典中词汇概率
        # attention层
        self.atteniton = Attention()
        # attention结果转换线性层
        self.atteniton_fc = nn.Linear(hidden_dim * 4, hidden_dim*2)

    def forward(self, token_seq, hidden_state, enc_output):
        # token_seq: [batch_size, seq_len]
        # embedded: [batch_size, seq_len, emb_dim]
        embedded = self.embedding(token_seq)

        # outputs: [batch_size, seq_len, hidden_dim * 2]
        # hidden: [1, batch_size, hidden_dim * 2]
        dec_output, hidden = self.rnn(embedded, hidden_state.unsqueeze(0))

        # attention运算
        c_t = self.atteniton(enc_output, dec_output)
        # [attention, dec_output]
        cat_output = torch.cat((c_t, dec_output), dim=-1)
        # 线性运算
        out = torch.tanh(self.atteniton_fc(cat_output))

        # logits: [batch_size, seq_len, input_dim]
        logits = self.fc(out)
        return logits, hidden

class Seq2Seq(nn.Module):

    def __init__(self,
                 enc_emb_size, 
                 dec_emb_size,
                 emb_dim,
                 hidden_size,
                 dropout=0.5,
                 ):
        
        super().__init__()

        # encoder
        self.encoder = Encoder(enc_emb_size, emb_dim, hidden_size, dropout=dropout)
        # decoder
        self.decoder = Decoder(dec_emb_size, emb_dim, hidden_size, dropout=dropout)


    def forward(self, enc_input, dec_input):
        # encoder last hidden state
        encoder_state, enc_output = self.encoder(enc_input)
        output,hidden = self.decoder(dec_input, encoder_state, enc_output)

        return output,hidden

In [10]:
from torch.nn.utils.rnn import pad_sequence
def get_proc(enc_voc, dec_voc):

    # 嵌套函数定义
    # 外部函数变量生命周期会延续到内部函数调用结束 （闭包）

    def batch_proc(data):
        """
        批次数据处理并返回
        """
        enc_ids, dec_ids, labels = [],[],[]
        for enc,dec in data:
            # token -> token index
            enc_idx = [enc_voc[tk] for tk in enc]
            dec_idx = [dec_voc[tk] for tk in dec]

            # encoder_input
            enc_ids.append(torch.tensor(enc_idx))
            # decoder_input
            dec_ids.append(torch.tensor(dec_idx[:-1]))
            # label
            labels.append(torch.tensor(dec_idx[1:]))

        
        # 数据转换张量 [batch, max_token_len]
        # 用批次中最长token序列构建张量
        enc_input = pad_sequence(enc_ids, batch_first=True)
        dec_input = pad_sequence(dec_ids, batch_first=True)
        targets = pad_sequence(labels, batch_first=True)

        # 返回数据都是模型训练和推理的需要
        return enc_input, dec_input, targets

    # 返回回调函数
    return batch_proc   

In [11]:
import pickle
import torch
import json
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

device = torch.device('cuda')

# 加载训练数据
with open('/kaggle/input/couplet/vocab.bin','rb') as f:
    evoc,dvoc = pickle.load(f)

with open('/kaggle/input/couplet/encoder.json') as f:
    enc_data = json.load(f)
with open('/kaggle/input/couplet/decoder.json') as f:
    dec_data = json.load(f)

ds = list(zip(enc_data,dec_data))
dl = DataLoader(ds, batch_size=256, shuffle=True, collate_fn=get_proc(evoc, dvoc))

# 构建训练模型
# 模型构建
model = Seq2Seq(
    enc_emb_size=len(evoc),
    dec_emb_size=len(dvoc),
    emb_dim=100,
    hidden_size=120,
    dropout=0.5,
)
model.to(device)

# 优化器、损失
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# 训练
for epoch in range(20):
    model.train()
    tpbar = tqdm(dl)
    for enc_input, dec_input, targets in tpbar:
        enc_input = enc_input.to(device)
        dec_input = dec_input.to(device)
        targets = targets.to(device)

        # 前向传播 
        logits, _ = model(enc_input, dec_input)

        # 计算损失
        # CrossEntropyLoss需要将logits和targets展平
        # logits: [batch_size, seq_len, vocab_size]
        # targets: [batch_size, seq_len]
        # 展平为 [batch_size * seq_len, vocab_size] 和 [batch_size * seq_len]
        loss = criterion(logits.view(-1, logits.size(-1)), targets.view(-1))

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tpbar.set_description(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

torch.save(model.state_dict(), '/kaggle/working/seq2seq_state_add.bin')

Epoch 20, Loss: 1.2622: 100%|██████████| 3010/3010 [02:39<00:00, 18.91it/s]


In [ ]:
import torch
import pickle

# 加载训练好的模型和词典
state_dict = torch.load('/kaggle/working/seq2seq_state_add.bin')
with open('/kaggle/input/couplet/vocab.bin','rb') as f:
    evoc,dvoc = pickle.load(f)

model = Seq2Seq(
    enc_emb_size=len(evoc),
    dec_emb_size=len(dvoc),
    emb_dim=100,
    hidden_size=120,
    dropout=0.5,
)
model.load_state_dict(state_dict)

# 创建解码器反向字典
dvoc_inv = {v:k for k,v in dvoc.items()}

def test(in_file):
    with open(in_file,'r',encoding='utf-8') as f, open('/kaggle/working/test_out_add.txt','w',encoding='utf-8') as w:
        lines = f.read().split('\n')
        for line in lines:
            #空数据排除
            if line == '':
                continue
            # 用户输入
            enc_idx = torch.tensor([[evoc.get(tk, evoc['UNK']) for tk in line.split(' ')[:-1]]])

            # 推理
            # 最大解码长度=输入长度
            max_dec_len = len(line.split(' ')[:-1])

            model.eval()
            with torch.no_grad():
                # 编码器
                hidden_state, enc_outputs = model.encoder(enc_idx)

                # 解码器输入 shape [1,1]
                dec_input = torch.tensor([[dvoc['BOS']]])

                # 循环decoder
                dec_tokens = []
                while True:
                    if len(dec_tokens) >= max_dec_len:
                        break
                    # 解码器 
                    # logits: [1,1,dec_voc_size]
                    # logits,hidden_state = model.decoder(dec_input, hidden_state)
                    logits,hidden_state = model.decoder(dec_input, hidden_state, enc_outputs)
                    
                    # 下个token index
                    next_token = torch.argmax(logits, dim=-1)

                    if dvoc_inv[next_token.squeeze().item()] == 'EOS':
                        break
                    # 收集每次token_index 【解码集合】
                    dec_tokens.append(next_token.squeeze().item())
                    # decoder的下一个输入 = token_index
                    dec_input = next_token
                    hidden_state = hidden_state.view(1, -1)

            # 输出解码结果
            w.write(''.join([dvoc_inv[tk] for tk in dec_tokens]))
            w.write('\n')
    return

/tmp/ipykernel_31/3848950086.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/kaggle/working/seq2seq_state_add.bin')


In [ ]:
test('/kaggle/input/testdate/in.txt')